In [2]:
import pandas as pd
import numpy as np
import pickle
import os

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# VOYAGE_API_KEY = os.getenv("VOYAGE_API_KEY")

In [3]:
movies_summary_df = pd.read_csv('../movie_summary_25_words.csv')
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [4]:
movies_summary_df.shape

(3883, 5)

In [5]:
from collections import defaultdict
from datetime import datetime
import pandas as pd
import json
import numpy as np
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [6]:
test_text = movies_summary_df.iloc[1873]['summary'] + " Genre: " + movies_summary_df.iloc[1873]['cleaned_genre']
print(test_text)

"All the King's Men" (1949) is a gripping political drama that explores corruption and power through the rise and fall of a charismatic Southern politician. Genre: Drama


In [7]:
movies_summary_df['all_text'] = movies_summary_df.apply(lambda x: x['summary'] + " Genre: " + x['cleaned_genre'], axis = 1)
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary,all_text
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ...","""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm...","""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram...","""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...,Father of the Bride Part II (1995) is a heartw...


### Using SFR-Embedding-Mistral for generating embeddings (Best on METB Leaderboard)
https://huggingface.co/Salesforce/SFR-Embedding-Mistral

In [17]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

In [18]:
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral')
tokenizer.add_eos_token = True

tokenizer_config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
import time

max_length = 4096
input_texts = test_text

start = time.time()
batch_dict = tokenizer(input_texts, max_length=max_length - 1, padding=True, truncation=True, return_tensors="pt")
outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
print("Time Taken:", time.time() - start)
# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
print(len(embeddings[0]))

Time Taken: 5.457363605499268
4096


### OpenAI text-embedding-3-large
Actual size is 3072 but as trained using MRL, can be used to decrease the size to 1024 or 512 for faster retrieval

In [8]:
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary,all_text
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ...","""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm...","""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram...","""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...,Father of the Bride Part II (1995) is a heartw...


In [12]:
def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [17]:
test_emb = get_embedding(test_text)
print(len(test_emb), test_emb)

3072 [0.017666401341557503, 0.02884022332727909, -0.007462936919182539, -0.010191701352596283, -0.014667111448943615, 0.014772968366742134, -0.015960922464728355, -0.00695129344239831, 3.72384165530093e-05, 0.036791279911994934, -0.023617930710315704, 0.028252126649022102, -0.030933843925595284, 0.02818155474960804, 0.015643350780010223, 0.012220632284879684, -0.02135964296758175, 0.01961887814104557, 0.021959500387310982, 0.00904491450637579, 0.02143021486699581, -0.03923775628209114, -0.0024214847944676876, 0.011409060098230839, 0.01409077737480402, 0.0023553241044282913, -0.022935740649700165, 0.02260640636086464, 0.03359203785657883, 0.03441537171602249, -0.013208633288741112, 0.01270287111401558, 0.03695594519376755, 0.006868959870189428, 0.03907309100031853, 0.004257814027369022, -0.0019907045643776655, 0.009897653944790363, -0.00819217599928379, 0.012420585379004478, 0.004096087999641895, 0.014525968581438065, -0.012514680624008179, 0.004969410132616758, 0.01980707049369812, -0.

In [20]:
movies_summary_df['embeddings_3072'] = movies_summary_df.all_text.apply(lambda x: get_embedding(x, model = "text-embedding-3-large"))

In [21]:
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary,all_text,embeddings_3072
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...,Toy Story is a groundbreaking animated film th...,"[-0.030682744458317757, 0.0427243672311306, -0..."
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ...","""Jumanji (1995) is a thrilling adventure film ...","[0.009568397887051105, 0.02835703268647194, -0..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm...","""Grumpier Old Men is a hilarious and heartwarm...","[0.02002841793000698, 0.010674105025827885, -0..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram...","""Waiting to Exhale"" (1995) is a heartfelt dram...","[-0.04223187267780304, 0.039355866611003876, -..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...,Father of the Bride Part II (1995) is a heartw...,"[-0.004295275546610355, 0.0179799385368824, -0..."


In [26]:
len(movies_summary_df['embeddings_3072'][465])

3072

In [27]:
def reduceEmbedding1024(emb):
    def normalize_l2(x):
        x = np.array(x)
        if x.ndim == 1:
            norm = np.linalg.norm(x)
            if norm == 0:
                return x
            return x / norm
        else:
            norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
            return np.where(norm == 0, x, x / norm)

    cut_dim = emb[:1024]
    norm_dim = normalize_l2(cut_dim)
    return norm_dim

In [29]:
print(len(reduceEmbedding1024(test_emb)))

1024


In [30]:
movies_summary_df['embeddings_1024'] = movies_summary_df.embeddings_3072.apply(lambda x: reduceEmbedding1024(x))

In [31]:
movies_summary_df.head()

,movie_id,name,cleaned_genre,year,summary,all_text,embeddings_3072,embeddings_1024
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...,Toy Story is a groundbreaking animated film th...,"[-0.030682744458317757, 0.0427243672311306, -0...","[-0.04189223359988252, 0.05833308604075958, -0..."
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ...","""Jumanji (1995) is a thrilling adventure film ...","[0.009568397887051105, 0.02835703268647194, -0...","[0.013023682103039223, 0.03859715946740607, -0..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm...","""Grumpier Old Men is a hilarious and heartwarm...","[0.02002841793000698, 0.010674105025827885, -0...","[0.02791051785617025, 0.014874854317657128, -0..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram...","""Waiting to Exhale"" (1995) is a heartfelt dram...","[-0.04223187267780304, 0.039355866611003876, -...","[-0.058955313685819216, 0.05494043513368762, -..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...,Father of the Bride Part II (1995) is a heartw...,"[-0.004295275546610355, 0.0179799385368824, -0...","[-0.005895226078259542, 0.02467729983744119, -..."


In [33]:
len(movies_summary_df['embeddings_1024'][465])

1024

In [34]:
movies_summary_df.to_csv('movies_summary_embeddings.csv', index=False)

In [35]:
df = pd.read_csv('movies_summary_embeddings.csv')
df.head()

,movie_id,name,cleaned_genre,year,summary,all_text,embeddings_3072,embeddings_1024
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...,Toy Story is a groundbreaking animated film th...,"[-0.030682744458317757, 0.0427243672311306, -0...",[-0.04189223 0.05833309 -0.02476214 ... -0.02...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ...","""Jumanji (1995) is a thrilling adventure film ...","[0.009568397887051105, 0.02835703268647194, -0...",[ 0.01302368 0.03859716 -0.01630003 ... -0.01...
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm...","""Grumpier Old Men is a hilarious and heartwarm...","[0.02002841793000698, 0.010674105025827885, -0...",[ 0.02791052 0.01487485 -0.02803959 ... -0.01...
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram...","""Waiting to Exhale"" (1995) is a heartfelt dram...","[-0.04223187267780304, 0.039355866611003876, -...",[-0.05895531 0.05494044 -0.01637648 ... -0.00...
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...,Father of the Bride Part II (1995) is a heartw...,"[-0.004295275546610355, 0.0179799385368824, -0...",[-0.00589523 0.0246773 -0.02444115 ... 0.00...


In [40]:
movies_summary_df['embeddings_1024'][0][0]

-0.04189223359988252